In [ ]:
%matplotlib nbagg
import bisect

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import plotly.subplots as sp


def second_derivative_magnitude(theta_deg, offset=0, gamma=2):
    """
    Return the magnitude of the second derivative of cos^2(2*theta)
    with respect to theta, evaluated at theta_deg (in degrees).
    The exact expression is 8 * |cos(4*theta)|.
    """
    # Convert degrees to radians for np.cos
    theta_rad = np.radians(theta_deg)
    return 8 * ((1 - offset) * abs(np.cos(4.0 * theta_rad)) ** gamma + offset)


def build_cdf_on_0_to_45(num_samples=20000):
    """
    Build a high-resolution CDF of w(theta) = second_derivative_magnitude(theta)
    over the fundamental domain [0, 45) degrees. Return (thetas, cdf_vals, total).

    - thetas[i]: the i-th sample angle in [0, 45].
    - cdf_vals[i]: integral of w from 0 up to thetas[i].
    - total: the integral of w from 0 to 45.
    """
    thetas = [45.0 * i / (num_samples - 1) for i in range(num_samples)]
    w_vals = [second_derivative_magnitude(t) for t in thetas]

    cdf_vals = [0.0] * num_samples
    for i in range(1, num_samples):
        dt = thetas[i] - thetas[i - 1]
        cdf_vals[i] = cdf_vals[i - 1] + 0.5 * (w_vals[i] + w_vals[i - 1]) * dt

    total = cdf_vals[-1]
    return thetas, cdf_vals, total


def inv_cdf(u, thetas, cdf_vals):
    idx = bisect.bisect_left(cdf_vals, u)
    if idx == 0:
        return 0.0
    if idx >= len(cdf_vals):
        return 45.0
    c0, c1 = cdf_vals[idx - 1], cdf_vals[idx]
    t0, t1 = thetas[idx - 1], thetas[idx]
    frac = 0.0 if (c1 == c0) else (u - c0) / (c1 - c0)
    return t0 + frac * (t1 - t0)


def generate_fundamental_angles(M, thetas_0_45, cdf_0_45, total_0_45):
    if M <= 1:
        return [0.0]
    step = total_0_45 / (M - 1)
    angles = [inv_cdf(i * step, thetas_0_45, cdf_0_45) for i in range(M)]
    return [min(a, 44.999999) for a in angles]


def generate_angles(theta_min_deg, theta_max_deg, N):
    thetas_0_45, cdf_0_45, total_0_45 = build_cdf_on_0_to_45()

    def count_points_for_M(M):
        base_angles = generate_fundamental_angles(M, thetas_0_45, cdf_0_45, total_0_45)
        angles_all = set()
        k_start = int(np.floor(theta_min_deg / 45.0)) - 1
        k_end = int(np.floor(theta_max_deg / 45.0)) + 1
        for k in range(k_start, k_end + 1):
            block_lo = 45.0 * k
            block_hi = 45.0 * (k + 1)
            if block_hi < theta_min_deg or block_lo > theta_max_deg:
                continue
            shifted = [a + 45.0 * k for a in base_angles if (a + 45.0 * k) % 45 != 0]
            clipped = [
                x for x in shifted if (x >= theta_min_deg and x <= theta_max_deg)
            ]
            angles_all.update(clipped)
        return sorted(angles_all)

    lo, hi = 1, max(1, N)
    best_M, best_diff, best_result = 1, float("inf"), [theta_min_deg]
    while lo <= hi:
        mid = (lo + hi) // 2
        angles_mid = count_points_for_M(mid)
        diff = abs(len(angles_mid) - N)
        if diff < best_diff:
            best_diff, best_M, best_result = diff, mid, angles_mid
            if diff == 0:
                break
        if len(angles_mid) < N:
            lo = mid + 1
        else:
            hi = mid - 1
    return np.array(best_result)


if __name__ == "__main__":
    theta_min_deg = -60
    theta_max_deg = 60
    N = 31
    angles = generate_angles(theta_min_deg, theta_max_deg, N)
    angles = angles.round(3)
    angles_string = ", ".join([str(round(angle, 3)) for angle in angles])

    print(len(angles))

    # Plot results
    Ex = np.cos(np.radians(angles))
    Ey = np.sin(np.radians(angles))
    # E_major = [max(abs(Ex[i]), abs(Ey[i])) for i in range(len(Ex))]
    E_major = np.maximum(np.abs(Ex), np.abs(Ey))
    E_minor = np.minimum(np.abs(Ex), np.abs(Ey))
    ellipticity = E_minor / E_major
    handedness = Ex * Ey
    peak_intensity = E_major ** 2
    min_intensity = E_minor ** 2

    df = pd.DataFrame(
            {
                "Angle": angles,
                "Ellipticity": ellipticity,
                "Peak Intensity": peak_intensity,
                "Min Intensity": min_intensity,
                "Handedness": [
                    "Linear" if abs(h) < 0.001 else "Right" if h > 0 else "Left"
                    for h in handedness
                ],
                "Ex": Ex,
                "Ey": Ey,
                "Local Density": second_derivative_magnitude(angles),
            }
    )
    display(df)

    fig = px.scatter(
            df,
            x="Angle",
            y="Ellipticity",
            symbol="Handedness",
            symbol_map={
                "Linear": "circle",
                "Right": "triangle-up",
                "Left": "triangle-down",
            },
    )

    fig.add_trace(
            go.Scatter(
                    x=df["Angle"],
                    y=df["Ex"] ** 2,
                    mode="lines+markers",
                    name="$I_x$",
                    line=dict(color="red", width=1),
                    marker=dict(color="red", size=5, symbol="circle"),
            )
    )
    fig.add_trace(
            go.Scatter(
                    x=df["Angle"],
                    y=df["Ey"] ** 2,
                    mode="lines+markers",
                    name="$I_y$",
                    line=dict(color="green", width=1),
                    marker=dict(color="green", size=5, symbol="circle"),
            )
    )
    #
    # fig.add_trace(
    #     go.Scatter(
    #         x=df['Angle'],
    #         y=second_derivative_magnitude(df['Angle'])/max(second_derivative_magnitude(df['Angle'])),
    #         mode='lines+markers',
    #         name='Sample Density',
    #         line=dict(color='purple', width=1),
    #         marker=dict(color='purple', size=5, symbol='circle')))

    fig.show()
    display(angles_string)
    fig_2 = sp.make_subplots(specs=[[{"secondary_y": True}]])
    theoretical_density = df["Local Density"] / np.sum(df["Local Density"]) * N
    print(sum(theoretical_density), sum(1 / np.gradient(df["Angle"])))
    print(
            np.mean(theoretical_density),
            np.mean(1 / np.gradient(df["Angle"])),
            N / (theta_max_deg - theta_min_deg),
    )
    fig_2.add_trace(
            go.Scatter(
                    x=df["Angle"],
                    y=theoretical_density,
                    mode="lines+markers",
                    name="Theoretical Density",
                    line=dict(color="purple", width=1),
                    marker=dict(color="purple", size=5, symbol="circle"),
            ),
            secondary_y=False,
    )
    local_derivative = np.gradient(df["Angle"])
    fig_2.add_trace(
            go.Scatter(
                    x=df["Angle"],
                    y=1 / local_derivative,
                    mode="lines+markers",
                    name="True Density",
                    line=dict(color="orange", width=1),
                    marker=dict(color="orange", size=5, symbol="circle"),
            ),
            secondary_y=False,
    )

    fig_2.add_trace(
            go.Scatter(
                    x=df["Angle"],
                    y=local_derivative,
                    mode="lines+markers",
                    name="Spacing",
                    line=dict(color="blue", width=1),
                    marker=dict(color="blue", size=5, symbol="circle"),
            ),
            secondary_y=True,
    )
    # adjust the y-axis range
    fig_2.update_yaxes(title_text="Density", secondary_y=False)
    fig_2.update_yaxes(title_text="Spacing", secondary_y=True, range=[0, 20])
    fig_2.show()